In [2]:
import sys
import os
BASE_DIR = os.path.abspath(os.path.join('..'))  # jeśli notebook w Strategies/notebook
sys.path.append(BASE_DIR)

import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import os
from datetime import datetime
import time
import config

def get_live_data(symbol, timeframe, candle_lookback):

    if not mt5.initialize():
        raise RuntimeError(f"MT5 initialize failed: {mt5.last_error()}")

    if not mt5.symbol_select(symbol, True):
        mt5.symbol_select(symbol, False)  # Deselect
        time.sleep(0.5)
        if not mt5.symbol_select(symbol, True):
            raise RuntimeError(f"Still can't select symbol: {symbol}")

    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, candle_lookback)

    if rates is None or len(rates) == 0:
        raise ValueError("Brak danych dla podanego zakresu dat.")

    df = pd.DataFrame(rates)

    df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)


    return df

def pandas_freq_from_timeframe(tf: str) -> str:
    mapping = {
        'H1': '1h',
        'H4': '4h',
        'D1': '1d',
        'M1': '1min',
        'M5': '5min',
        'M15': '15min',
    }
    return mapping.get(tf.upper(), tf)

def get_data(symbol, timeframe, start_date, end_date):
    """
    Pobiera dane z MetaTrader 5 dla wybranego symbolu i przedziału czasowego.

    Args:
        symbol (str): np. 'EURUSD'
        timeframe (mt5.TIMEFRAME_*): np. mt5.TIMEFRAME_H1
        start_date (datetime): data początkowa
        end_date (datetime): data końcowa

    Returns:
        pandas.DataFrame: dane OHLC + wolumen z datami
    """
    # Inicjalizacja
    if not mt5.initialize():
        raise RuntimeError(f"MT5 initialize failed: {mt5.last_error()}")

    # Próba włączenia symbolu
    if not mt5.symbol_select(symbol, True):
        time.sleep(0.5)
        if not mt5.symbol_select(symbol, True):
            mt5.shutdown()
            raise RuntimeError(f"Nie można wybrać symbolu: {symbol}")

    # Pobranie danych
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if rates is None or len(rates) == 0:
        mt5.shutdown()
        raise ValueError(f"Brak danych dla {symbol} w podanym zakresie dat.")

    # Konwersja do DataFrame
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df[['time', 'open', 'high', 'low', 'close', 'tick_volume']]

    # Zakończenie połączenia
    mt5.shutdown()

    return df

def merge_informative_data(df: pd.DataFrame, timeframe: str, informative_df: pd.DataFrame) -> pd.DataFrame:
    freq = pandas_freq_from_timeframe(timeframe)
    time_col = f'time_{timeframe}'

    #print(f"[merge_informative_data] Before timezone fix, df['time'] sample: {df['time'].head(3).tolist()}")
    if df['time'].dt.tz is None:
        #print("[merge_informative_data] df['time'] is tz-naive, localizing...")
        df['time'] = df['time'].dt.tz_localize(config.SERVER_TIMEZONE)
    else:
        #print(f"[merge_informative_data] df['time'] is tz-aware with tz: {df['time'].dt.tz}, converting...")
        df['time'] = df['time'].dt.tz_convert(config.SERVER_TIMEZONE)
    #print(f"[merge_informative_data] After fix, df['time'] sample: {df['time'].head(3).tolist()}")

    df[time_col] = df['time'].dt.tz_convert(config.SERVER_TIMEZONE).dt.floor(freq)
    #print(f"[merge_informative_data] Created column '{time_col}' sample: {df[time_col].head(3).tolist()}")


    informative_df = informative_df.rename(columns={
        col: f"{col}_{timeframe}" for col in informative_df.columns if col != 'time'
    })

    merged = df.merge(
        informative_df,
        left_on=time_col,
        right_on='time',
        how='left'
    )
    #print(f"[merge_informative_data] Merged dataframe length: {len(merged)}")

    return merged.drop(columns=['time'], errors='ignore')



data = get_data("EURUSD", mt5.TIMEFRAME_M5, datetime(2025,1,1), datetime(2025,10,20))
data_H1 = get_data("EURUSD", mt5.TIMEFRAME_H1, datetime(2025,1,1), datetime(2025,10,20))

In [12]:
import pandas as pd
from utils.decorators import informative
from TechnicalAnalysis.Indicators import indicators as qtpylib
from TechnicalAnalysis.PointOfInterestSMC.core import SmartMoneyConcepts
from TechnicalAnalysis.SessionsSMC.core import SessionsSMC

import talib.abstract as ta

def get_informative_dataframe(symbol, timeframe: str, startup_candle_count: int) -> pd.DataFrame:
    freq = pandas_freq_from_timeframe(timeframe)
    tf_minutes = pd.to_timedelta(freq).total_seconds() / 60
    extra_minutes = tf_minutes * startup_candle_count

    start_time = pd.to_datetime(config.TIMERANGE['start']).tz_localize(config.SERVER_TIMEZONE) - pd.to_timedelta(extra_minutes, unit='m')
    end_time = pd.to_datetime(config.TIMERANGE['end']).tz_localize(config.SERVER_TIMEZONE)



    df = get_live_data(
        symbol,
        getattr(mt5, f"TIMEFRAME_{timeframe}"),
        6000
    )


    return df

def populate_informative_indicators(obj_with_df_and_symbol):
    for attr_name in dir(obj_with_df_and_symbol):
        attr = getattr(obj_with_df_and_symbol, attr_name)
        if callable(attr) and getattr(attr, '_informative', False):
            timeframe = attr._informative_timeframe
            if timeframe not in obj_with_df_and_symbol.informative_dataframes:
                informative_df = get_informative_dataframe(
                    symbol=obj_with_df_and_symbol.symbol,
                    timeframe=timeframe,
                    startup_candle_count=obj_with_df_and_symbol.startup_candle_count
                )
                informative_df = attr(df=informative_df.copy())
                obj_with_df_and_symbol.informative_dataframes[timeframe] = informative_df
            else:
                informative_df = obj_with_df_and_symbol.informative_dataframes[timeframe]

            obj_with_df_and_symbol.df = merge_informative_data(
                obj_with_df_and_symbol.df,
                timeframe,
                informative_df
            )

import re
from collections import defaultdict

def merge_signals(signal_list):
    if not signal_list:
        return None
    direction = signal_list[0][0]
    reasons = sorted(set(sig[1] for sig in signal_list))
    merged_reason = "_".join(reasons)
    return (direction, merged_reason)

def merge_levels(level_list, direction="long", close_price=None):
    if not level_list:
        return None

    min_distance_ratio = 0.0005  # 0.05%
    max_sl_ratio = 0.003         # 0.3%
    min_sl_ratio = 0.001         # 0.1%

    # 🔹 Zbierz unikalne tagi (dla informacji)
    tags = [level.get("tag", "?") for level in level_list]
    combined_tag = "_".join(sorted(set(tags)))

    sl_all, tp1_all, tp2_all = [], [], []

    # 🔹 Zbierz wszystkie poziomy SL/TP z level["extra"]
    for level in level_list:
        extra = level.get("extra", {}) or {}
        sl_val = extra.get("sl")
        tp1_val = extra.get("tp1")
        tp2_val = extra.get("tp2")

        if sl_val is not None:
            sl_all.append((level["source"], sl_val))
        if tp1_val is not None:
            tp1_all.append((level["source"], tp1_val))
        if tp2_val is not None:
            tp2_all.append((level["source"], tp2_val))

    # 🔹 Uzupełnij brakujące poziomy (na podstawie ceny close)
    if close_price is not None:
        if not sl_all:
            sl_value = close_price - (close_price * min_distance_ratio * 1.2) if direction == "long" else close_price + (close_price * min_distance_ratio * 1.2)
            sl_all.append(("auto", sl_value))
        if not tp1_all:
            tp1_value = close_price + (close_price * min_distance_ratio * 2) if direction == "long" else close_price - (close_price * min_distance_ratio * 2)
            tp1_all.append(("auto", tp1_value))
        if not tp2_all:
            tp2_value = close_price + (close_price * min_distance_ratio * 3) if direction == "long" else close_price - (close_price * min_distance_ratio * 3)
            tp2_all.append(("auto", tp2_value))

    # 🔹 Wybierz finalne wartości
    if direction == "long":
        sl_final = min(sl_all, key=lambda x: x[1])
        tp1_final = max(tp1_all, key=lambda x: x[1])
        tp2_final = max(tp2_all, key=lambda x: x[1])
    else:
        sl_final = max(sl_all, key=lambda x: x[1])
        tp1_final = min(tp1_all, key=lambda x: x[1])
        tp2_final = min(tp2_all, key=lambda x: x[1])

    # 🔹 Walidacja odległości SL / TP względem ceny close
    if close_price is not None:
        risk = abs(close_price - sl_final[1])
        max_allowed_risk = close_price * max_sl_ratio
        min_sl = close_price * min_sl_ratio

        # Zbyt mały SL → wymuś minimalny dystans
        if risk < min_sl:
            new_sl_price = close_price - min_sl if direction == "long" else close_price + min_sl
            sl_final = ("min_0.1%", new_sl_price)

        # RR check i korekta TP
        risk = abs(close_price - sl_final[1])
        reward_tp1 = abs(tp1_final[1] - close_price)
        reward_tp2 = abs(tp2_final[1] - close_price)

        # Wyrównaj TP1/TP2 dla lepszego RR
        if reward_tp1 / risk < 2:
            new_tp1 = close_price + risk * 2 if direction == "long" else close_price - risk * 2
            tp1_final = ("RR_1:2", new_tp1)

        if reward_tp2 / risk < 4:
            new_tp2 = close_price + risk * 4 if direction == "long" else close_price - risk * 4
            tp2_final = ("RR_1:4", new_tp2)

        if reward_tp1 / risk > 3:
            new_tp1 = close_price + risk * 3 if direction == "long" else close_price - risk * 3
            tp1_final = ("RR_1:3", new_tp1)

        if reward_tp2 / risk > 6:
            new_tp2 = close_price + risk * 6 if direction == "long" else close_price - risk * 6
            tp2_final = ("RR_1:6", new_tp2)

    return (
        ("SL", sl_final[1], f"SL_{sl_final[0]}_{combined_tag}"),
        ("TP", tp1_final[1], f"TP1_{tp1_final[0]}_{combined_tag}"),
        ("TP", tp2_final[1], f"TP2_{tp2_final[0]}_{combined_tag}")
    )




class Poi:
    def __init__(self, df: pd.DataFrame, symbol, startup_candle_count: int = 600):
        self.startup_candle_count = startup_candle_count
        self.df = df.copy()
        self.symbol = symbol
        self.informative_dataframes = {}
        # Inicjalizacja klasy SmartMoneyConcepts
        
        self.smc = SmartMoneyConcepts(self.df)
        self.sessions = SessionsSMC(self.df)
        self.sessions_h1 = None

    @informative('H1')
    def populate_indicators_H1(self, df: pd.DataFrame):

        df['idx'] = df.index
        df['atr'] = ta.ATR(df, 14)


        # Aktualizujemy niezależne instancje
        self.smc.df = df.copy()
        self.smc.find_validate_zones(tf="H1")

        self.sessions_h1 = SessionsSMC(df.copy())
        self.sessions_h1.df = self.sessions_h1.calculate_previous_ranges()

        # Zwracamy coś, by merge mógł zadziałać
        return df

    def populate_indicators(self):
        self.df = self.df.rename(columns={'time_x': 'time'})
        if 'time_y' in self.df.columns:
            self.df = self.df.drop(columns=['time_y'])

        self.df['idx'] = self.df.index
        self.df['atr'] = ta.ATR(self.df, 14)
        heikinashi = qtpylib.heikinashi(self.df)
        self.df[['ha_open', 'ha_close', 'ha_high', 'ha_low']] = heikinashi[['open', 'close', 'high', 'low']]

        self.df['candle_bullish'] = (
            qtpylib.candlestick_confirmation(self.df, 'bullish')
        )
        self.df['candle_bearish'] = (
            qtpylib.candlestick_confirmation(self.df, 'bearish')
        )

        first_high = self.df['high'].shift(2)
        first_low = self.df['low'].shift(2)

        self.df['min_5'] = self.df['low'].rolling(5).min()
        self.df['max_5'] = self.df['high'].rolling(5).max()

        cisd_bull_cond = ((self.df['high'] < first_low))
        cisd_bear_cond = ((self.df['low'] > first_high))

        self.df.loc[cisd_bull_cond, 'cisd_bull_line'] = first_low
        self.df.loc[cisd_bear_cond, 'cisd_bear_line'] = first_high

        self.df[f'cisd_bull_line'] = self.df[f'cisd_bull_line'].ffill()
        self.df[f'cisd_bear_line'] = self.df[f'cisd_bear_line'].ffill()

        # Aktualizujemy również na M5
        self.smc.df = self.df.copy()
        self.smc.find_validate_zones(tf="M5")
        self.smc.detect_reaction()

        self.sessions.df = self.df.copy()
        self.sessions.calculate_sessions_ranges()

        if self.sessions_h1 is not None:
            self.sessions.df = pd.merge_asof(
                self.sessions.df.sort_values('time'),
                self.sessions_h1.df.sort_values('time'),
                on='time',
                direction='backward',
                suffixes=('', '_H1')
            )

        self.sessions.detect_session_type()
        self.sessions.detect_signals()

    def merge_external_dfs(self):
        """
        Łączy dane z:
        - self.smc.df
        - self.sessions.df
        - sygnały z self.sessions.detect_signals()
    
        Pomija kolumny już obecne w self.df.
        """
        base = self.df.copy()
    
        # --- Łączenie z self.smc.df ---
        if hasattr(self, "smc") and hasattr(self.smc, "df"):
            smc_df = self.smc.df.copy()
            new_cols = [c for c in smc_df.columns if c not in base.columns]
            if new_cols:
                base = base.merge(smc_df[['time'] + new_cols], on='time', how='left', validate='1:1')
    
        # --- Łączenie z self.sessions.df ---
        if hasattr(self, "sessions") and hasattr(self.sessions, "df"):
            sessions_df = self.sessions.df.copy()
            new_cols = [c for c in sessions_df.columns if c not in base.columns]
            if new_cols:
                base = base.merge(sessions_df[['time'] + new_cols], on='time', how='left', validate='1:1')

        
    
        self.df = base

        print(f"Kolumny self.df: {list(self.df.columns)}")

    def calculate_levels(self, signals, close):

        if not isinstance(signals, dict):
            return None

        direction = signals.get("direction")
        tag = signals.get("tag")

        risk = close * 0.001  # np. 0.1%
        rr1 = 2
        rr2 = 4

        if direction == "long":
            sl = close - risk
            tp1 = close + risk * rr1
            tp2 = close + risk * rr2
        else:
            sl = close + risk
            tp1 = close - risk * rr1
            tp2 = close - risk * rr2

        return {
            "SL": {"level": sl, "tag": "auto"},
            "TP1": {"level": tp1, "tag": "RR_1:2"},
            "TP2": {"level": tp2, "tag": "RR_1:4"},
        }

    def populate_entry_trend(self):
        """
        Buduje sygnały wejścia łączące:
        - kierunek sesyjny (sessions_signal)
        - kierunek dnia (prev_day_direction)
        - bias rynkowy (session_bias)
        - strefy HTF/LTF (OB, FVG, Breaker)
        """

        df = self.df.copy()

        # --- 🔹 1. Agregacja stref ---
        def merge_flags(prefix):
            return df[f"{prefix}_reaction_H1"] | df[f"{prefix}_in_zone_H1"], \
                   df[f"{prefix}_reaction"] | df[f"{prefix}_in_zone"]

        for side in ["bullish", "bearish"]:
            for zone in ["breaker", "fvg", "ob"]:
                df[f"{side}_{zone}_H1"], df[f"{side}_{zone}"] = merge_flags(f"{side}_{zone}")

        # --- 🔹 2. Listy aktywnych stref ---
        def active_cols(df, side, timeframe):
            cols = [f"{side}_breaker{timeframe}", f"{side}_ob{timeframe}", f"{side}_fvg{timeframe}"]
            return df[cols].apply(lambda x: [col.split("_")[1].upper() for col in x.index if x[col]], axis=1)

        df["htf_long_active"] = active_cols(df, "bullish", "_H1")
        df["ltf_long_active"] = active_cols(df, "bullish", "")
        df["htf_short_active"] = active_cols(df, "bearish", "_H1")
        df["ltf_short_active"] = active_cols(df, "bearish", "")

        # --- 🔹 3. Bias i kierunek dnia ---
        df["prev_day_direction"] = np.where(df["prev_close"] > df["prev_open"], "bullish",
                                            np.where(df["prev_close"] < df["prev_open"], "bearish", None))
        df["session_bias"] = np.where(df["close"] > df["PDH"], "bullish",
                                      np.where(df["close"] < df["PDL"], "bearish", "neutral"))

        # --- 🔹 4. Inicjalizacja sygnałów ---
        df["signal_entry"] = None
        df["signal_strength"] = 0.0

        # --- 🔹 5. Maski logiczne ---
        long_mask = (
                (df["sessions_signal"] == "long")
                #(df["session_bias"] == "bullish") &
                #(df["prev_day_direction"] == "bullish") &
                #(df["htf_long_active"].apply(len) > 0) &
                #(df["ltf_long_active"].apply(len) > 0)
        )

        short_mask = (
                (df["sessions_signal"] == "short")
                #(df["session_bias"] == "bearish") &
                #(df["prev_day_direction"] == "bearish") &
                #(df["htf_short_active"].apply(len) > 0) &
                #(df["ltf_short_active"].apply(len) > 0)
        )

        # --- 🔹 6. Generowanie sygnałów + scoring ---
        def build_entry(row, direction):
            htf = row[f"htf_{direction}_active"]
            ltf = row[f"ltf_{direction}_active"]
            score = len(htf) + len(ltf)
            tag = f"{row['session_context']}__{row['session_bias']}__{direction.upper()}__HTF:{'-'.join(htf)}__LTF:{'-'.join(ltf)}"
            return {"direction": direction, "tag": tag}, score

        df.loc[long_mask, ["signal_entry", "signal_strength"]] = df.loc[long_mask].apply(
            lambda r: build_entry(r, "long"), axis=1, result_type="expand"
        )

        df.loc[short_mask, ["signal_entry", "signal_strength"]] = df.loc[short_mask].apply(
            lambda r: build_entry(r, "short"), axis=1, result_type="expand"
        )

        # --- 🔹 7. Poziomy SL/TP ---
        has_signals = df["signal_entry"].apply(bool)
        df.loc[has_signals, "levels"] = df.loc[has_signals].apply(
            lambda row: self.calculate_levels(row["signal_entry"], row["close"]),
            axis=1
        )

        print("here")
        print(df[df["signal_entry"].notna()])

        print("sessions_signal counts:")
        print(df["sessions_signal"].value_counts(dropna=False))
        
        # 2. Ile wierszy spełnia podstawowe warunki (poszczególne elementy maski)?
        print("count sessions_signal == 'long':", (df["sessions_signal"] == "long").sum())
        print("count sessions_signal == 'short':", (df["sessions_signal"] == "short").sum())
        
        # 3. Sprawdź htf/ltf — czy rzeczywiście zwracają listy i jakie są ich długości
        print("htf_long_active non-empty:", (df["htf_long_active"].apply(lambda x: len(x) if isinstance(x, (list,tuple)) else 0) > 0).sum())
        print("ltf_long_active non-empty:", (df["ltf_long_active"].apply(lambda x: len(x) if isinstance(x, (list,tuple)) else 0) > 0).sum())
        print("htf_short_active non-empty:", (df["htf_short_active"].apply(lambda x: len(x) if isinstance(x, (list,tuple)) else 0) > 0).sum())
        print("ltf_short_active non-empty:", (df["ltf_short_active"].apply(lambda x: len(x) if isinstance(x, (list,tuple)) else 0) > 0).sum())
        
        # 4. Sprawdź biasy i kierunek poprzedniego dnia
        print("prev_day_direction unique:", df["prev_day_direction"].unique())
        print("session_bias unique:", df["session_bias"].unique())
        
        # 5. Pokaż pierwsze N wierszy gdzie sessions_signal nie jest NaN (jeśli w ogóle są)
        print(df[df["sessions_signal"].notna()].head(10))

        self.df = df
        return df

    def populate_exit_trend(self):

        df = self.df

        df['signal_exit'] = None

        



    def get_bullish_zones(self):
        return []

    def get_bearish_zones(self):
        return []

    def get_extra_values_to_plot(self):
        return [
            #("london_high", self.sessions.df["london_main_high"], "blue", "dot"),
            #("london_low", self.sessions.df["london_main_low"], "blue", "dot"),
            #("asia_high", self.sessions.df["asia_main_high"], "purple", "dot"),
            #("asia_low", self.sessions.df["asia_main_low"], "purple", "dot"),
            #("ny_high", self.sessions.df["ny_main_high"], "orange", "dash"),
            #("ny_low", self.sessions.df["ny_main_low"], "orange", "dash"),

            #("PDH", self.sessions.df["PDH"], "blue"),
            #("PDL", self.sessions.df["PDL"], "blue"),

            #("PWH", self.sessions.df["PWH"], "yellow"),
            #("PWL", self.sessions.df["PWL"], "yellow"),
        ]

    def get_bullish_zones(self):
        return [
            #("Bullish IFVG H1", self.smc.bullish_ifvg_validated_H1, "rgba(255, 160, 122, 0.7)"),
            # Pomarańcz (pozostawiony bez zmian)
            # ("Bullish IFVG", self.bullish_ifvg_validated, "rgba(139, 0, 0, 1)"),

            #("Bullish FVG H1", self.smc.bullish_fvg_validated_H1, "rgba(255, 152, 0, 0.7)"),  # Jasnoniebieski
            # ("Bullish FVG", self.bullish_fvg_validated, "rgba(255, 152, 0, 0.7)"),             # Ciemnoniebieski

            ("Bullish OB H1", self.smc.bullish_ob_validated_H1, "rgba(144, 238, 144, 0.7)"),  # Jasnozielony
            # ("Bullish OB", self.bullish_ob_validated, "rgba(0, 100, 0, 1)"),           # Ciemnozielony

            ("Bullish Breaker H1", self.smc.bullish_breaker_validated_H1, "rgba(173, 216, 230, 0.7)"),  # Jasnoniebieski
            # ("Bullish Breaker", self.bullish_breaker_validated, "rgba(0, 0, 139, 1)"),             # Ciemnoniebieski

            # ("Bullish GAP ", self.bullish_gap_validated, "rgba(56, 142, 60, 1)"),
        ]

    def get_bearish_zones(self):
        return [
            # ("Bearish Breaker", self.smc.bearish_breaker_validated, "rgba(64, 64, 64, 1)"),      # Ciemnoszary
             ("Bearish Breaker H1", self.smc.bearish_breaker_validated_H1, "rgba(169, 169, 169, 0.7)"),  # Jasnoszary

            # ("Bearish OB", self.smc.bearish_ob_validated, "rgba(139, 0, 0, 1)"),                # Ciemnoczerwony
             ("Bearish OB H1", self.smc.bearish_ob_validated_H1, "rgba(255, 160, 122, 0.7)"),       # Jasnoczerwony

            # ("Bearish IFVG H1", self.smc.bearish_ifvg_validated_H1, "rgba(139, 0, 0, 1)"),  # Pomarańcz (pozostawiony bez zmian)
            # ("Bearish IFVG", self.smc.bearish_ifvg_validated, "rgba(255, 160, 122, 0.7)"),

            # ("Bearish FVG", self.smc.bearish_fvg_validated, "rgba(0, 0, 139, 1)"),      # Ciemnoszary
            # ("Bearish FVG H1", self.smc.bearish_fvg_validated_H1, "rgba(173, 216, 230, 0.7)"),  # Jasnoszary
        ]
    
    def bool_series(self):
        return []


    def run(self) -> pd.DataFrame:


        timings = []  # Lista do przechowywania czasów

        def timeit(label, func):
            start = time.time()
            func()
            end = time.time()
            duration = end - start
            timings.append((label, duration))
            #print(f"{label} finished in {duration:.4f} seconds")

        timeit("_populate_informative_indicators", lambda: populate_informative_indicators(self))
        timeit("self.populate_indicators()", lambda: self.populate_indicators())
        timeit("self.merge_external_dfs()", lambda: self.merge_external_dfs())
        timeit("self.populate_entry_trend()", lambda: self.populate_entry_trend())

        # 3️⃣ Zwróć końcowy DataFrame z M5 + H1 scalonymi danymi
        print("\n⏱️ Profil czasu wykonania:")
        for label, duration in timings:
            print(f"   {label:<40} {duration:.3f}s")



        return self.sessions.df

In [13]:
from backtesting.plot import plot_trades_with_indicators

poi = Poi(df=data, symbol="EURUSD")


df_bt = poi.run()



#print("\n📊 === INFORMACJE O self.smc.df ===")
#print(f"Kształt: {df_bt.shape}")
#print(f"Kolumny: {list(df_bt.columns)}")
#print("\nPrzykładowe dane:")
#print(df_bt.head(5))

         
plot_trades_with_indicators(
    df_bt,
    "EURUSD",
    bullish_zones=poi.get_bullish_zones(),
    bearish_zones=poi.get_bearish_zones(),
    extra_series=poi.get_extra_values_to_plot(),
    bool_series=poi.bool_series(),
                )


Kolumny self.df: ['time', 'open', 'high', 'low', 'close', 'tick_volume', 'time_H1', 'open_H1', 'high_H1', 'low_H1', 'close_H1', 'tick_volume_H1', 'spread_H1', 'real_volume_H1', 'idx_H1', 'atr_H1', 'idx', 'atr', 'ha_open', 'ha_close', 'ha_high', 'ha_low', 'min_5', 'max_5', 'cisd_bull_line', 'cisd_bear_line', 'candle_form', 'candle_bullish', 'candle_bearish', 'bullish_fvg_in_zone', 'bullish_fvg_reaction', 'bullish_fvg_in_zone_H1', 'bullish_fvg_reaction_H1', 'bullish_ob_in_zone', 'bullish_ob_reaction', 'bullish_ob_in_zone_H1', 'bullish_ob_reaction_H1', 'bullish_breaker_in_zone', 'bullish_breaker_reaction', 'bullish_breaker_in_zone_H1', 'bullish_breaker_reaction_H1', 'bullish_ifvg_in_zone', 'bullish_ifvg_reaction', 'bullish_ifvg_in_zone_H1', 'bullish_ifvg_reaction_H1', 'bearish_fvg_in_zone', 'bearish_fvg_reaction', 'bearish_fvg_in_zone_H1', 'bearish_fvg_reaction_H1', 'bearish_ob_in_zone', 'bearish_ob_reaction', 'bearish_ob_in_zone_H1', 'bearish_ob_reaction_H1', 'bearish_breaker_in_zone', '

AttributeError: 'str' object has no attribute 'iterrows'